In [ ]:
import pandas as pd
import numpy as np

cols = ['ID', 'Age', 'Gender', 'Education', 'Country', 'Ethnicity', 'N-Score', 'E-Score', 'O-Score', 'A-Score', 'C-Score', 'Impulsive', 'ImpSS', 'Alcohol', 'Amphet',
        'Amyl', 'Benzos', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms', 'Nicotine', 'Semer', 'VSA']

df = pd.read_csv('./data/drug_consumption.data', names=cols)
df.pop('ID')
df.head()
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVR
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

cols = ['ID', 'Age', 'Gender', 'Education', 'Country', 'Ethnicity', 'N-Score', 'E-Score', 'O-Score', 'A-Score', 'C-Score', 'Impulsive', 'ImpSS', 'Alcohol', 'Amphet',
        'Amyl', 'Benzos', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms', 'Nicotine', 'Semer', 'VSA']

df = pd.read_csv('./data/drug_consumption.data', names=cols)
df.pop('ID')

feature_cols = df.columns[:12]
target_cols = df.columns[12:]

for col in target_cols:
    df[col] = df[col].apply(lambda x: 0 if x == 'CL0' else 1)

X = df[feature_cols].values
y = df[target_cols].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Define the objective function for Optuna optimization
def objective(trial):
    C = trial.suggest_loguniform('C', 1e-3, 10)
    epsilon = trial.suggest_loguniform('epsilon', 1e-3, 1)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    
    svr = SVR(C=C, epsilon=epsilon, kernel=kernel)
    model = OneVsRestClassifier(svr)
    
    return -cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_squared_error').mean()

# Optuna optimization
study = optuna.create_study()
study.optimize(objective, n_trials=50)

# Train the model with best parameters
best_params = study.best_params
svr = SVR(**best_params)
model = OneVsRestClassifier(svr)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)

for i, col in enumerate(target_cols):
    print(f"Classification Report for {col}:\n")
    print(classification_report(y_test[:, i], y_pred[:, i], zero_division=1))
    print("-" * 50)
